#Setup Data + Build Query Engine Tools

#Setup Base ReAct Agent (gpt-3.5-turbo)

This (not trained) ReAct agent can sometimes enter the incorrect reasoning loop to answer the question.

In [2]:
# !pip install llama_index

In [3]:
from llama_index.core.agent import ReActAgent

In [4]:
from llama_index.llms.openai import OpenAI

setup llm and agent using base llm

In [2]:
llm_35 = OpenAI(model="gpt-3.5-turbo-0613")
llm4 = OpenAI(model="gpt-4-0613")
# base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

NameError: name 'OpenAI' is not defined

In [7]:
# import os
# from dotenv import load_dotenv

# load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY_QUERY_DECOMP")


Bad response below

In [ ]:
response = base_agent.chat(
    "Analyze Uber revenue growth over the last few quarters"
)

print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'revenue growth'}
Observation: The revenue growth for the three months ended March 31, 2022, compared to the same period in 2021, was 136%. This increase in revenue was primarily driven by an increase in Gross Bookings of 35%, or 39% on a constant currency basis. The growth in revenue was also attributed to increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19 and a $1.5 billion increase in Freight revenue resulting from the acquisition of Transplace. Additionally, there was a $304 million increase in Delivery revenue and a $200 million increase in Mobility revenue due to business model changes in the UK.
Thought: I can answer without using any more tools.
Answer: The revenue growth for Uber over the last few quarters has been significant. In the three months ended March 31, 2022, compared to the same period in 2021, the revenue growth was 136%. Th

bad response (x2)

In [ ]:
response = base_agent.chat(
    "Can you tell me about the risk factors in the quarter with the highest"
    " revenue growth?"
)
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'risk factors'}
Observation: The company faces various risk factors that could have an adverse effect on its business, financial condition, and results of operations. These risk factors include the negative impact of economic, social, weather, and regulatory conditions on its operations, the potential failure or inferior performance of autonomous vehicle technologies, the need to retain and attract high-quality personnel, the risk of security breaches and data privacy breaches, the potential harm from cyberattacks, the impact of climate change risks, the reliance on third parties for distribution and software, the need for additional capital, the risks associated with identifying and integrating suitable businesses, the limitations on providing services in certain jurisdictions, the legal and regulatory risks, the risks related to payment and financial services regulations, the risk

#Generate Training/Eval Questions

- Generate a synthetic dataset of questions to ask (later comes from visual Q&A dataset).
- generate an initial set of questions over a “base” document (the March 2022 10Q)
- use an LLM to generate variations of that question that can apply across multiple quarters (later apply across many types of sorroundings).

Aim: stress-test the LLM reasoning capabilities.

In [10]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [3]:
# base_question_gen_query = (
#     "You are a Teacher/ Professor. Your task is to setup a quiz/examination."
#     " Using the provided context from the Computer Vision document, formulate a"
#     " single question that is in the format of the questions in the document."
#     " The question should be in the same format but pertain to a classroom "
#     " environment. "
# )

# dataset_generator = RagDatasetGenerator.from_documents(
#     cv_docs,
#     question_gen_query=base_question_gen_query,
#     llm=llm_35
# )

Need to fix dataset generator - these questions will be from the CV & location Q&A dataset

In [9]:
import json

file_path = 'cv_class_questions.json'
with open(file_path, 'r') as file_cv:
    data = json.load(file_cv)

cv_questions = data

In [10]:
file_path = 'loc_questions.json'
with open(file_path, 'r') as file_loc:
    data = json.load(file_loc)

loc_questions = data

In [11]:
questions = cv_questions + loc_questions
# for question in questions:
#     print(question)


print(len(questions))

40


Variations of each question

In [25]:
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate


vary_question_tmpl = """\
You are a query assistant. Given a question pertaining to CV or location, your goal
is to generate up to {num_vary} variations of that question that might span multiple depths.

This can include compare/contrasting different objects/requests, replacing the current subject with
another subject, or generating questions that can only be answered by synthesizing multiple of the base questions (be creative!)

You are given a valid set of cv and location questions. Please only generate question variations that can be
answered in that set.

For example:
Base Question: "Is the student typing on a laptop?
Valid 10Qs: [cv_questions, loc_questions]
Question Variations:
Is the student working on a productive project or slacking off using their laptop?
Contrast the activities of the left student's laptop use from the right student's use?
Is the teacher typing on a laptop?

Now let's give it a shot!

Base Question: {base_question}
Valid 10Qs: {valid_10qs}
Question Variations:
"""


def gen_question_variations(base_questions, num_vary=3):
    """Generate question variations."""

    VALID_10Q_STR = "[cv_questions, loc_questions]"

    llm = llm_35
    prompt_tmpl = PromptTemplate(vary_question_tmpl)

    new_questions = []
    for idx, question in enumerate(base_questions):
        new_questions.append(question)
        response = llm.complete(
            prompt_tmpl.format(
                num_vary=num_vary,
                base_question=question,
                valid_10qs=VALID_10Q_STR,
            )
        )
        # parse into newlines
        raw_lines = str(response).split("\n")
        cur_new_questions = [l for l in raw_lines if l != ""]
        print(f"[{idx}] Original Question: {question}")
        print(f"[{idx}] Generated Question Variations: {cur_new_questions}")
        new_questions.extend(cur_new_questions)

    return new_questions


def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")


def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [3]:
import json

In [4]:
file_path = 'cv_loc_enhanced_questions.json'
with open(file_path, 'r') as file_loc:
    data = json.load(file_loc)

new_questions = data

In [7]:
new_questions = list(set(new_questions))
len(new_questions)

80

60 training questions, 20 evaluating questions

In [ ]:
# IMPORTANT: Train/Eval on every possible 60/20 split
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [ ]:
save_questions(train_questions, "train_questions_10q.txt")
# save_questions(eval_questions, "eval_questions_10q.txt")

In [ ]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

#Log Input/Output Pairs

- run the train questions through a ReAct agent (GPT-4) to collect prompt outputs
- Every prompt call to the LLM is logged as an input/output pair.
- OpenAIFineTuningHandler automatically collects prompt input/outputs when agent queries are run.
- This dataset can then be saved, in a dataset format .jsonl to directly feed to the OpenAI Finetuning endpoints to finetune 3.5 on 4 questions

create finetuning_handler + callback_manager
Also set context window length

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.finetuning.callbacks import OpenAIFineTuningHandler
from llama_index.core.callbacks import CallbackManager
from llama_index.core.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

from llama_index.core import Settings

# limit the context window artifically to test refine process
Settings.context_window = 2048

Setup gpt-4 ReAct Agent

In [ ]:
llm = OpenAI(model="gpt-4-0613")
gpt4_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [ ]:
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = gpt4_agent.query(question)
    print(f"[{idx}] Agent Response: {str(response)}")

[0] Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices of Uber Technologies, Inc. are not explicitly mentioned in the provided context information.
Thought: The first tool did not provide the needed information. I should try the other tools.
Action: june_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices of Uber Technologies, Inc. are not explicitly mentioned in the provided context information.
Thought: The second tool also did not provide the needed information. I should try the last tool.
Action: sept_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices of Uber Technologies, Inc. are not explicitly mentioned in the provided context information.
Thought: I cannot an

Save events

In [ ]:
finetuning_handler.save_finetuning_events("finetuning_events_10q.jsonl")

Wrote 179 examples to finetuning_events_10q.jsonl


#Create OpenAIFinetuneEngine
the finetune engine will launch a finetuning job, and returning an LLM model that you can directly plugin to the rest of LlamaIndex workflows.

If finetuning the same model even further - list job id below

In [ ]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events_10q.jsonl",
    #start_job_id="<start-job-id>"  # if you have an existing job, can specify id here
)

finetune

In [ ]:
finetune_engine.finetune()

Num examples: 179
First example:
{'role': 'system', 'content': '\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: sept_2022\nTo

get finetuning job id

In [ ]:
finetune_engine.get_current_job()

FineTuningJob(id='ftjob-qiGoHWTJZfc6iun19unXc2pV', created_at=1709675066, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-by7b0ByCCb8VyGQAQud5kEuU', result_files=[], status='queued', trained_tokens=None, training_file='file-Ii4qXOwTGc4VMrScvv0iZacL', validation_file=None, user_provided_suffix=None)

get finetuned llm - see status on OAI dev portal (must be successful)

In [ ]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

#Compare Finetuned Agent vs. Base Agent
- run some sample queries from the evaluation dataset over both our finetuned agent as well as the base agent.
- qualitatively look at their abilities to perform chain of thought prompting in order to arrive at the right answer.

Setup FT agent using ft model

In [ ]:
ft_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=ft_llm,
    callback_manager=callback_manager,
    verbose=True,
)

retrieve evaluation questions

In [ ]:
eval_questions = []
with open("eval_questions_10q.txt", "r") as f:
    for line in f:
        eval_questions.append(line.strip())

In [ ]:
qidx = 0
print(eval_questions[qidx])

What is the total fair value of Uber's financial assets as of March 31, 2022?


Get response from base agent
base agent = normal gpt-3.5

In [ ]:
base_response = base_agent.query(eval_questions[qidx])
print(str(base_response))

Thought: To find the total fair value of Uber's financial assets as of March 31, 2022, I can use the "march_2022" tool.
Action: march_2022
Action Input: {'input': 'financial assets'}
Observation: The financial assets mentioned in the context include non-marketable equity securities, marketable equity securities, and a note receivable from a related party. These assets are measured at fair value on a recurring basis and are categorized into different levels of the fair value hierarchy. Level 1 assets are valued based on quoted market prices, Level 2 assets are obtained from readily available pricing sources or models using market observable inputs, and Level 3 assets are valued based on unobservable inputs and other estimation techniques. The Level 3 assets primarily consist of non-marketable equity securities and note receivables from privately held companies. The fair value of these assets is estimated based on the investee's financing transactions, such as the issuance of preferred s

get response from ft agent
ft agent = finetuned gpt 3.5 on answers from gpt 4.0

In [ ]:
ft_response = ft_agent.query(eval_questions[qidx])
print(str(ft_response))

Thought: I need to use the march_2022 tool to get the information about Uber's financial assets as of March 31, 2022.
Action: march_2022
Action Input: {'input': 'financial assets'}
Observation: The financial assets mentioned in the context include non-marketable equity securities, marketable equity securities, and a note receivable from a related party. These assets are measured at fair value on a recurring basis and are categorized into different levels of the fair value hierarchy. Level 1 assets are valued based on quoted market prices, Level 2 assets are obtained from readily available pricing sources or models using market observable inputs, and Level 3 assets are valued based on unobservable inputs and other estimation techniques. The Level 3 assets primarily consist of non-marketable equity securities and the note receivable from a related party. The fair value of these assets is estimated using various valuation techniques, including the guideline public company approach and opt

#TL;DR
- finetuned model does much better than the base model in terms of reasoning about the current sequence of steps @same costs.
- It passes more detailed answers to the downstream tools
- Is more capable of refining its approach when initial queries don’t work.